In [1]:
# Requires brainvisa

In [2]:
import anatomist.api as anatomist
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from soma import aims
import json
import sys
import pandas as pd
import scipy
import scipy.stats
import numpy as np
import glob

In [4]:
a = anatomist.Anatomist()

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module meshsplit
loading module palettecontrols
loading module volumepalettes


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [5]:
Rspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_right/meshes/Rspam_model_meshes_1.arg"

Lspam_model = "/casa/host/build/share/brainvisa-share-5.2/models/models_2008/descriptive_models/segments/global_registered_spam_left/meshes/Lspam_model_meshes_1.arg"


json_regions = "/neurospin/dico/data/deep_folding/current/sulci_regions_gridsearch.json"

path_summary = "/neurospin/dico/data/deep_folding/current/models/Champollion_V0/UKB-RAP/right-handed-ukb40"
path_file = "all_regions.csv"


In [6]:
with open(json_regions) as f:
    regions = json.load(f)

In [7]:
file_to_display = f"{path_summary}/{path_file}"

In [8]:
next(iter(regions['brain']))

'S.C.-sylv._left'

In [9]:
df = pd.read_csv(file_to_display)[["region", "p"]]
df["p"] = -np.log10(df["p"])
df.head()

,region,p
0,SPoC_left,3.291979
1,STsbr_right,2.004995
2,SC-SPoC_left,4.641480
3,FCLp-SGSM_right,3.748429
4,STsbr_right,0.276057


In [10]:

res = df.groupby(['region']).mean()
res["side"] = res.index.str.split('_').str[-1]
res = res.reset_index()

In [11]:
res = res[~res.region.str.contains("fronto-parietal")]
res = res[~res.region.str.contains("STs-SGSM")]
res = res[~res.region.str.contains("FCLp-SGSM")]
res = res[~res.region.str.contains("CINGULATE")]

In [12]:
"CINGULATE_left".replace("CINGULATE", "CINGULATE.")

'CINGULATE._left'

In [13]:
def get_sulci(region):
    region = region.replace("CINGULATE", "CINGULATE.")
    region = region.replace("ORBITAL", "S.Or.")
    list_sulci = list(regions['brain'][f"{region}"].keys())
    list_sulci = [x.replace("paracingular.", "S.F.int.") for x in list_sulci]
    return list_sulci

In [14]:
get_sulci("CINGULATE_left")

['S.F.int._left', 'F.C.M.ant._left']

In [15]:
# res["region"].tolist()

In [16]:
dic = {
    "FCLp-subsc-FCLa-INSULA": "F.C.L.p.-subsc.-F.C.L.a.-INSULA.",
    "FCMpost-SpC": "F.C.M.post.-S.p.C.",
    "FColl-SRh": "F.Coll.-S.Rh.",
    "FIP": "F.I.P.",
    "FPO-SCu-ScCal": "F.P.O.-S.Cu.-Sc.Cal.",
    "Lobule_parietal_sup": "Lobule_parietal_sup.",
    "ORBITAL": "S.Or.",
    "SC-SPeC": "S.C.-S.Pe.C.",
    "SC-SPoC": "S.C.-S.Po.C.",
    "SC-sylv": "S.C.-sylv.",
    "SFinf-BROCA-SPeCinf": "S.F.inf.-BROCA-S.Pe.C.inf.",
    "SFint-FCMant": "S.F.int.-F.C.M.ant.",
    "SFint-SR": "S.F.int.-S.R.",
    "SFinter-SFsup": "S.F.inter.-S.F.sup.",
    "SFmarginal-SFinfant":"S.F.marginal-S.F.inf.ant.",
    "SFmedian-SFpoltr-SFsup": "S.F.median-S.F.pol.tr.-S.F.sup.",
    "SOr-SOlf": "S.Or.-S.Olf.",
    "SPeC": "S.Pe.C.",
    "SPoC": "S.Po.C.",
    "STi-SOTlat": "S.T.i.-S.O.T.lat.",
    "STi-STs-STpol": "S.T.i.-S.T.s.-S.T.pol.",
    "STsbr": "S.T.s.br.",
    "STs": "S.T.s.",
    "ScCal-SLi": "Sc.Cal.-S.Li.",
    "SsP-SPaint": "S.s.P.-S.Pa.int.",
}

In [17]:
for key in dic.keys():
    res["region"] = res["region"].str.replace(key, dic[key])
res.head()

,region,p,side
4,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,3.737555,left
5,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,5.209588,right
6,F.C.M.post.-S.p.C._left,1.479360,left
7,F.C.M.post.-S.p.C._right,1.173353,right
8,F.Coll.-S.Rh._left,8.196886,left


In [18]:
res['sulcus'] = res.apply(lambda x: get_sulci(x.region), axis=1)
res.head()

,region,p,side,sulcus
4,F.C.L.p.-subsc.-F.C.L.a.-INSULA._left,3.737555,left,"[F.C.L.p._left, F.C.L.r.sc.ant._left, F.C.L.r...."
5,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,5.209588,right,"[F.C.L.p._right, F.C.L.r.sc.ant._right, F.C.L...."
6,F.C.M.post.-S.p.C._left,1.479360,left,"[F.C.M.post._left, S.C.LPC._left, S.p.C._left]"
7,F.C.M.post.-S.p.C._right,1.173353,right,"[F.C.M.post._right, S.C.LPC._right, S.p.C._right]"
8,F.Coll.-S.Rh._left,8.196886,left,"[F.Coll._left, S.Rh._left]"


In [19]:
res = res.sort_values(by="p", ascending=False)
res = res.explode("sulcus")
res[res.region.str.contains("S.T.s.")]

,region,p,side,sulcus
46,S.T.i.-S.T.s.-S.T.pol._left,3.757519,left,S.T.i.ant._left
46,S.T.i.-S.T.s.-S.T.pol._left,3.757519,left,S.T.i.post._left
46,S.T.i.-S.T.s.-S.T.pol._left,3.757519,left,S.T.s._left
46,S.T.i.-S.T.s.-S.T.pol._left,3.757519,left,S.T.pol._left
51,S.T.s._right,3.216023,right,S.T.s._right
47,S.T.i.-S.T.s.-S.T.pol._right,2.684765,right,S.T.i.ant._right
47,S.T.i.-S.T.s.-S.T.pol._right,2.684765,right,S.T.i.post._right
47,S.T.i.-S.T.s.-S.T.pol._right,2.684765,right,S.T.s._right
47,S.T.i.-S.T.s.-S.T.pol._right,2.684765,right,S.T.pol._right
52,S.T.s.br._left,1.282853,left,F.I.P.r.int.1_left


In [20]:
for _, row in res.iterrows():
    print(row.sulcus)

F.Coll._left
S.Rh._left
F.C.L.p._right
F.C.L.r.sc.ant._right
F.C.L.r.sc.post._right
INSULA_right
F.C.L.a._right
F.Coll._right
S.Rh._right
F.C.M.ant._left
S.F.int._left
S.C._right
S.C.sylvian._right
S.T.i.ant._left
S.T.i.post._left
S.T.s._left
S.T.pol._left
F.C.L.p._left
F.C.L.r.sc.ant._left
F.C.L.r.sc.post._left
INSULA_left
F.C.L.a._left
S.F.inf._right
S.F.inf.ant._right
F.C.L.r.ant._right
F.C.L.r.asc._right
F.C.L.r.diag._right
S.Pe.C.inf._right
S.Or._right
S.Olf._right
F.P.O._left
S.Pa.t._left
S.Cu._left
F.Cal.ant.-Sc.Cal._left
S.O.p._left
F.I.P._right
S.Po.C.sup._right
S.Pa.sup._right
S.C._left
S.C.sylvian._left
S.Po.C.sup._left
F.I.P.Po.C.inf._left
F.C.L.r.retroC.tr._left
S.T.s._right
S.s.P._left
S.Pa.int._left
S.s.P._right
S.Pa.int._right
S.C._right
S.C.sylvian._right
S.Pe.C.inf._right
S.Pe.C.inter._right
S.Pe.C.marginal._right
S.Pe.C.sup._right
S.Pe.C.median._right
F.P.O._right
S.Pa.t._right
S.Cu._right
F.Cal.ant.-Sc.Cal._right
S.O.p._right
S.T.i.ant._right
S.T.i.post._right
S.O.T

In [21]:
res[res.sulcus=="S.F.orbitaire._right"]

,region,p,side,sulcus
35,S.F.marginal-S.F.inf.ant._right,2.145321,right,S.F.orbitaire._right


In [22]:
res

,region,p,side,sulcus
8,F.Coll.-S.Rh._left,8.196886,left,F.Coll._left
8,F.Coll.-S.Rh._left,8.196886,left,S.Rh._left
5,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,5.209588,right,F.C.L.p._right
5,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,5.209588,right,F.C.L.r.sc.ant._right
5,F.C.L.p.-subsc.-F.C.L.a.-INSULA._right,5.209588,right,F.C.L.r.sc.post._right
...,...,...,...,...
34,S.F.marginal-S.F.inf.ant._left,0.696752,left,S.F.orbitaire._left
34,S.F.marginal-S.F.inf.ant._left,0.696752,left,S.F.inf.ant._left
37,S.F.median-S.F.pol.tr.-S.F.sup._right,0.631705,right,S.F.median._right
37,S.F.median-S.F.pol.tr.-S.F.sup._right,0.631705,right,S.F.polaire.tr._right


In [23]:
def set_color_property(res, side):
    global dic

    if side == "L":
        spam_model_file = Lspam_model
    else:
        spam_model_file = Rspam_model
        
    dic[f"aims{side}"] = aims.read(spam_model_file)

    for vertex in dic[f"aims{side}"].vertices():
        vertex['p_value'] = 0.

    for _, row in res.iterrows():
        for vertex in dic[f"aims{side}"].vertices():
            vname = vertex.get('name')
            if vname == row.sulcus:
                    vertex['p_value'] = max(vertex['p_value'],
                                                row.p)
                    # if row.p < -np.log10(0.05/56):
                    #     vertex['p_value'] = 0.
                    # elif vertex['p_value'] != 0.: 
                    #     vertex['p_value'] = min(vertex['p_value'],
                    #                             row.p)
                    # else:
                    #     vertex['p_value'] = row.p
    
    dic[f"ana{side}"] = a.toAObject(dic[f"aims{side}"])

    dic[f"ana{side}"].setColorMode(dic[f"ana{side}"].PropertyMap)
    dic[f"ana{side}"].setColorProperty('p_value')
    dic[f"ana{side}"].notifyObservers()
    
                
def visualize_whole_hemisphere(view_quaternion, side, i):
    global block
    global dic
    try:
        block
    except NameError:
        block = a.createWindowsBlock(4)

    dic[f"win{i}"] = a.createWindow('3D',
                                    block=block,
                                    no_decoration=True,
                                    options={'hidden': 1})
    dic[f"win{i}"].addObjects(dic[f"ana{side}"])
    dic[f"ana{side}"].setPalette("green_yellow_red",
                              minVal=-np.log10(0.05/56), maxVal=10,
                              absoluteMode=True)
    
    dic[f"win{i}"].camera(view_quaternion=view_quaternion)

middle_view = [0.5, -0.5, -0.5, 0.5]
side_view = [0.5, 0.5, 0.5, 0.5]
bottom_view = [0, -1, 0, 0]
top_view = [0, 0, 0, -1]
 
def visualize_whole(res, side, start):
    set_color_property(res, side)
    visualize_whole_hemisphere(middle_view if side == "L" else side_view, side, start+0)
    visualize_whole_hemisphere(top_view, side, start+1)
    visualize_whole_hemisphere(bottom_view, side, start+2)
    visualize_whole_hemisphere(side_view if side == "L" else middle_view, side, start+3)


In [24]:
dic = {}

visualize_whole(res, "L", 0)
visualize_whole(res, "R", 4)

Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60
Reading FGraph version 2.0
bounding box found : -90, -80, -90
                     90, 120, 60


Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.
